## 测试mps态

In [1]:
import Library.TensorNetwork as TN
import torch as tc

In [2]:
mps = TN.rand_mps_pack(number=2, length=3, chi=4, phydim=2)
print(mps.get_norm())
print(mps.center)
print(TN.inner_mps_pack(mps, mps))

tensor([1.0000, 1.0000])
1
tensor([1.0000+0.j, 1.0000+0.j])


将大张量表示的量子态转化为mps态，并计算误差

In [3]:
number = 2
length = 3
dtype = tc.complex64
device = tc.device('cpu')
shape = [number, 2 ** length]
states = tc.randn(shape, dtype=dtype, device=device)
shape_ = [number] + [2]*length
norm = tc.sum(states * states.conj(), dim=1, keepdim=True)
states = states / tc.sqrt(norm)
states = states.reshape(shape_)
print(states.shape)
mps_ = TN.TensorTrain_pack(tensor_packs=[states], length=length, phydim=2, center=-1, chi=8, device=device, dtype=dtype, initialize=True)
mps_copy = TN.copy_from_mps_pack(mps_)
mps_copy.merge_all()
print(mps_copy.node_list[0].shape)
print(tc.dist(mps_copy.node_list[0].squeeze(), states))

torch.Size([2, 2, 2, 2])
torch.Size([2, 1, 2, 2, 2, 1])
tensor(3.3790e-07)


将n体门转化为mpo，并计算和转化前的误差

In [4]:
n = 3
shape = [2] * (n*2)
n_body_op = tc.rand(shape, dtype=dtype, device=device)
print(list(n_body_op.shape))
gate_list = TN.n_body_gate_to_mpo(n_body_op, n, dtype=dtype, device=device)
print('')
for gate in gate_list:
    print(gate.shape)
gate_U = tc.einsum('abcd, defg, ghij -> abehcfij', gate_list[0], gate_list[1], gate_list[2])
print(gate_U.squeeze().shape)
print(tc.dist(gate_U.squeeze(), n_body_op))

[2, 2, 2, 2, 2, 2]

torch.Size([1, 2, 2, 4])
torch.Size([4, 2, 2, 4])
torch.Size([4, 2, 2, 1])
torch.Size([2, 2, 2, 2, 2, 2])
tensor(0.)


In [7]:
number = 3
length = 5
dtype = tc.complex64
device = tc.device('cpu')
shape = [number, 2 ** length]
states = tc.randn(shape, dtype=dtype, device=device)
shape_ = [number] + [2]*length
norm = tc.sum(states * states.conj(), dim=1, keepdim=True)
states = states / tc.sqrt(norm)
states = states.reshape(shape_)
print(states.shape)
mps = TN.TensorTrain_pack(tensor_packs=[states], length=length, phydim=2, center=-1, chi=8, device=device, dtype=dtype, initialize=True)
n = 2
shape = [2] * (n*2)
n_body_op = tc.rand(shape, dtype=dtype, device=device)
print(list(n_body_op.shape))
gate_list = TN.n_body_gate_to_mpo(n_body_op, n, dtype=dtype, device=device)
states_ = states.reshape([number] + [2 ** n] + [-1])
gate_U = n_body_op.reshape([2 ** n, 2 ** n])
states_act_U = tc.einsum('ij, njk -> nik', gate_U, states_)
states_act_U = states_act_U.reshape([number] + [2]*length)
print()
print(states_act_U.shape)
mps_act_U = mps.act_two_body_gate(n_body_op, [0, 1])
mps_act_U.merge_all()
print(mps_act_U.node_list[0].shape)
print(tc.dist(mps_act_U.node_list[0].squeeze(), states_act_U))

torch.Size([3, 2, 2, 2, 2, 2])
[2, 2, 2, 2]

torch.Size([3, 2, 2, 2, 2, 2])
torch.Size([3, 1, 2, 2, 2, 2, 2, 1])
tensor(1.2196)
